In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from PIL import Image
from io import BytesIO
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
sio = socketio.Server()
app = Flask(__name__)

In [3]:
def send_control(steering_angle, throttle):
    data={'steering_angle': str(steering_angle).replace('.', ','),
                            'throttle': str(throttle).replace('.', ',') }
#     print('emitting', data)
    sio.emit("steer", data, skip_sid=True)

In [4]:
model = load_model('model.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 58, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    (None, 56, 76, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_______________________________________

In [5]:
def process_image(img):
    return img[10:130:2,::4,:]

In [ ]:
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"].replace(',','.'))
        image_str = data["image"]
        image_bytes = base64.b64decode(image_str)
        img = Image.open(BytesIO(image_bytes))
        img_array = np.asarray(img)
#         plt.imshow(img_array)
#         print(img_array.shape)
        img = process_image(img_array)
        img_batch = np.expand_dims(img, axis=0)
        steering_angle = float(model.predict(img_batch))
#         print(steering_angle)
        throttle = 0.15
        if speed < 15:
            throttle = 0.5
        if speed > 17:
            throttle = -0.1
        
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(4219) wsgi starting up on http://0.0.0.0:4567
(4219) accepted ('127.0.0.1', 46091)
127.0.0.1 - - [14/May/2019 22:53:59] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 20.678059
